### 쿠팡 스크래핑

#### HTTP METHOD
 - 겟방식 : 루트를 전부 보여주는 방식
  - 포스트 : BODY에 메세지를 담아 보내주는 방식

In [1]:
import requests
from bs4 import BeautifulSoup
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=1=4&backgroundColor="
# headers = {User-Agent : "~"} 를 입력하면 사람이 브라우저에 로그인 한 것으로 인식된다.

In [ ]:
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")
print(res.text)

In [ ]:
import re

items = soup.find_all("li", attrs = {"class":re.compile("^search-product")})
print(items[0].find("div", attrs={"class":"name"}).get_text()) # 정규식을 섞은 것으로
# li태그의 class속성이 search-product로 '시작하는'(정규식) text를 가져온다


In [ ]:
for item in items:
    
    # 광고제품은 제외할것
    
    ad_badge = item.find("span", attrs = {"class":"ad-badge-text"})
    if ad_badge:
        print("<광고 상품 제외>")
        continue
        
    name = item.find("div", attrs={"class":"name"}).get_text() # 정규식을 섞은 것으로
    
    # 애플 제품 제외
    
    if "Apple" in name:
        print("<애플 상품 제외>")
        continue
    
    price = item.find("strong", attrs = {"class":"price-vlaue"})
    
    # 리뷰 n개 이상, 평점은 4.5 이상만 조회
    
    rate = item.find("em", attrs = {"class":"rating"})
    if rate:
        rate = rate.get_text()
    else:
        print('<평점 없는 상품 제외>')
        continue
        
    rate_cnt = item.find("span", attrs = {"class" :"rating-total-count" })
   
    if rate_cnt:
        rate_cnt = rate_cnt.get_text()
    else:
        print("<평점 없는 상품 제외>")
        continue
        
    if float(rate) >= 4.5 and int(rate_cnt[1:-1]) >= 500:
        print(name, price, rate, rate_cnt)

### 1 ~ 6page까지 조건에 맞는 구문만 스크래이핑

In [ ]:
link = item.find("a", attrs = {"class":"search-product-link"})["href"]

for i in range(1, 6):
    url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=4&backgroundColor="
    ".format(i)
    
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    items = soup.find_all("li", attrs = {"class":re.compile("^search-product")})
    
    for item in items:
    
    # 광고제품은 제외할것
    
        ad_badge = item.find("span", attrs = {"class":"ad-badge-text"})
        if ad_badge:
            print("<광고 상품 제외>")
            continue
        
        name = item.find("div", attrs={"class":"name"}).get_text() # 정규식을 섞은 것으로
    
    # 애플 제품 제외
    
        if "Apple" in name:
            print("<애플 상품 제외>")
            continue
    
        price = item.find("strong", attrs = {"class":"price-vlaue"})
    
    # 리뷰 n개 이상, 평점은 4.5 이상만 조회
    
        rate = item.find("em", attrs = {"class":"rating"})
        if rate:
            rate = rate.get_text()
        else:
            print('<평점 없는 상품 제외>')
            continue
        
        rate_cnt = item.find("span", attrs = {"class" :"rating-total-count" })
   
        if rate_cnt:
            rate_cnt = rate_cnt.get_text()
        else:
            print("<평점 없는 상품 제외>")
            continue
        
        if float(rate) >= 4.5 and int(rate_cnt[1:-1]) >= 500:
            #print(name, price, rate, rate_cnt)
            print(f"제품명 : {name}")
            print(f"가격 : {price}")
            print(f"평점 : {rate}점 ({rate_cnt}개)")
            print("바로가기 : {}".format("https://www.coupang.com" + link))
            print("-"*100) # 줄 긋는거
            
    